Abstract

Neste artigo, apresentamos MidiNet, uma rede neural convoluta profunda (CNN)
Baseada na rede adversária generativa (GAN) que se destina a fornecer uma visão geral,
Estrutura de rede altamente adaptável para geração de música de domínio simbólico. o
A rede leva o ruído aleatório como entrada e gera uma seqüência de melodia uma medida
(Bar) após o outro. Além disso, tem um novo sub-modelo CNN reflexivo que
Nos permite orientar o processo de geração fornecendo não só 1D, mas também 2D
Condições. Em nossa implementação, usamos o acorde pretendido da barra atual
Como uma condição 1D para fornecer um contexto harmônico, e a melodia gerada para o
Antes da barra anteriormente como uma condição 2D para fornecer informações seqüenciais. o
A saída da rede é uma matriz de 16 por 128 cada vez, representando a presença de
Cada uma das 128 notas MIDI na sequência de melodia gerada dessa barra, com a
A menor unidade temporal é a décima sexta nota. MidiNet pode gerar música de
Número arbitrário de barras, concatenando estas 16 por 128 matrizes. A melodia
A sequência pode então ser reproduzida com um sintetizador. Nós fornecemos exemplos de clipes
Mostrando a eficácia da MidiNet na geração de música harmônica.

Introduction

As redes adversárias generativas (GANs) [1, 2] ganharam uma tremenda atenção nos últimos anos.
Em um GAN, um modelo de gerador e um modelo discriminador são treinados simultaneamente sob o
Conceito de minimax de dois jogadores de teoria dos jogos. O modelo do gerador visa produzir dados artificiais
Isso poderia enganar o modelo discriminador, enquanto a tarefa do modelo discriminador é distinguir
Entre dados autênticos e artificiais. Para a diversidade do resultado da geração, a entrada para o
O modelo do gerador geralmente é um ruído aleatório. O modelo do gerador em um GAN bem-sucedido pode converter
Um ruído aleatório dado em algo que parece ser realista para o ser humano. Pode ser uma imagem, texto
Passagem ou clipe de som, dependendo dos dados de treinamento. Até à data, a maioria dos trabalhos existentes tem
Focado na geração de imagens usando GANs [3-5].



Para modelar a natureza temporal da música, uma série de trabalhos anteriores sobre geração de música são baseados em
Redes neurais recorrentes (RNNs) [6-10]. Embora essas RNNs às vezes possam produzir
Música, o treinamento de uma RNN geralmente é mais lento, em comparação com o de um neural convolucional
Rede (CNN) [11]. Também é menos direto adicionar restrições ou reguladores a uma RNN para informar o processo de geração com conhecimento musical a priori, que pode ser essencial para a qualidade estética do resultado da geração.

O objetivo deste artigo é propor um modelo GAN novo para geração de música de domínio simbólico que
É baseado na CNN, com foco na geração da sequência de melodia principal.2
Em vez de gerar um
Sequência de melodia continuamente, propomos gerar uma melodia uma medida (barra) após a outra,
De forma sucessiva. Isso nos permite usar uma matriz 2D que represente a ocorrência de diferentes
Notas ao longo do tempo como entrada para uma CNN. Além disso, aproveitando a idéia de contraditório condicional
Treinamento [5, 11, 12], propomos um novo sub-modelo CNN reflexivo que usa a sequência de melodia
Gerado anteriormente para a barra anterior (que é novamente representada como uma matriz 2D) para condicionar
A geração para a barra atual. Desta forma, a dependência temporal entre as melodias de
As barras sucessivas podem ser modeladas sem uma unidade recorrente, como é usado nas RNNs. A CNN reflexiva
Realmente nos permite usar as seqüências de melodia de várias barras anteriores para condicionar o GAN, para
Modelo de dependências temporais de longo prazo na música. Além de tais condições 2D, certamente podemos
Use as condições 1D mais simples (por exemplo, rótulos de classe), bem como para condicionar o GAN. Em outras palavras,
Dependendo da informação a priori que possamos, podemos adicionar diferentes componentes ao nosso GAN. este
Estrutura altamente adaptativa pode ser útil em vista dos padrões temporais complexos e hierárquicos (em
Tempo) e relações harmônicas (entre a melodia principal e os sons simultâneos e acompanhantes)
Encontrado na música. Nos referimos a este novo modelo CNN-GAN como MidiNet.

Observamos que, enquanto as condições de 1D foram amplamente utilizadas no trabalho anterior no GAN [5, 11, 12], a
O uso de uma condição 2D não foi tentado antes, conforme nosso conhecimento. O proposto
CNN reflexiva é necessária para adicionar essa condição 2D às camadas intermediárias de uma CNN, cada uma das
Que tem tamanhos diferentes. Embora usemos tais condições 2D para a aplicação específica de símbolos simbólicos
Geração de música de domínio, a idéia é realmente genérica e é aplicável a outros problemas
também. Ele abre nova porta para incluir informações laterais aos GANs. Mais detalhes do método são
Descrito na Seção 2.

Em nossa implementação (consulte a Seção 3), reunimos 1.022 guias MIDI para treinar a MidiNet. Nós costumavamos
A informação da corda como a condição 1D, assumindo que a seqüência de corda desejada é dada e
Que há apenas um acorde por barra. Além disso, a partir da segunda barra, usamos a melodia
Sequência gerada para a barra anterior como condição 2D. Em outras palavras, condicionamos a
Geração da melodia para cada barra por um rótulo de corda e a melodia da barra anterior. Para
Estabilizar as atualizações de treinamento do GAN, usamos técnicas como a correspondência de recursos
E alinhamento de etiquetas unilateral [5]. MidiNet pode gerar música de número arbitrário de barras, por
Concatenando a sequência de melodia gerada para cada barra. Definimos o número alvo de barras para 8
E forneça exemplos do resultado da geração na Seção 4.

2.1 Generator model

Ao projetar CNNs para tarefas relacionadas à imagem, o uso de filtros convolucionais 2D para o aprendizado de recursos é comum [3,13]. Em contraste, para tarefas relacionadas ao áudio, às vezes é bom o suficiente para usar filtros convolucionais 1D, uma vez que os dados de áudio são seqüenciais [14, 15]. Portanto, usamos filtros da forma h-by-1 na última camada convolucional transposta, onde h indica o número de notas MIDI consideradas pela rede. Da mesma forma, a série de camadas convolucionais transpostas converte um vetor de valores aleatórios em uma matriz 2D da forma desejada h-by-w como a saída, o que corresponde à ocorrência de diferentes notas nas unidades temporais $w$. O tamanho dos filtros na primeira camada convolucional transposta é definido como 1-por-w. O modelo do gerador está ilustrado na Figura 1.

<img src="midinet.png"> 